___
# Ciência dos Dados - Projeto 3: Filmes#
___

### _Gabriela Caruso e Mariana Abrantes - 2C_ ###
___

## Objetivo ##

Desenvolver um programa que cria uma playlist de filmes similares a um outro usado como parâmetro.

___

## Imports

In [1]:
%matplotlib inline
%reset -f
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
#from emoji import UNICODE_EMOJI
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D

from sklearn import cluster, datasets
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler

np.random.seed(42)


## Lendo as tabelas

In [2]:
#df = pd.read_csv('https://query.data.world/s/dgju3rv2knocz2ost6qyly7vwyekcv')
#writer = pd.ExcelWriter('Projeto3_CD.xlsx')
#df.to_excel(writer,'data')
#writer.save()

In [3]:
df = pd.read_excel('Projeto3_CD.xlsx')
df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


## Eliminando colunas

In [24]:
dado = df.drop(labels=None, axis=0, index=None, columns='movie_imdb_link', level=None, inplace=False, errors='raise')
dado2 = dado.drop(labels=None, axis=0, index=None, columns='aspect_ratio', level=None, inplace=False, errors='raise')
data = dado2.drop(labels=None, axis=0, index=None, columns='language', level=None, inplace=False, errors='raise')
data.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,facenumber_in_poster,plot_keywords,num_user_for_reviews,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,0.0,avatar|future|marine|native|paraplegic,3054.0,USA,PG-13,237000000.0,2009.0,936.0,7.9,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,0.0,goddess|marriage ceremony|marriage proposal|pi...,1238.0,USA,PG-13,300000000.0,2007.0,5000.0,7.1,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,1.0,bomb|espionage|sequel|spy|terrorist,994.0,UK,PG-13,245000000.0,2015.0,393.0,6.8,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,0.0,deception|imprisonment|lawlessness|police offi...,2701.0,USA,PG-13,250000000.0,2012.0,23000.0,8.5,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,0


In [25]:
list(data)

['color',
 'director_name',
 'num_critic_for_reviews',
 'duration',
 'director_facebook_likes',
 'actor_3_facebook_likes',
 'actor_2_name',
 'actor_1_facebook_likes',
 'gross',
 'genres',
 'actor_1_name',
 'movie_title',
 'num_voted_users',
 'cast_total_facebook_likes',
 'actor_3_name',
 'facenumber_in_poster',
 'plot_keywords',
 'num_user_for_reviews',
 'country',
 'content_rating',
 'budget',
 'title_year',
 'actor_2_facebook_likes',
 'imdb_score',
 'movie_facebook_likes']

In [26]:
lista_nova = []
for color in data["color"]:
    if color == " Black and White":
        lista_nova.append(1)
    elif color == "Color":
        lista_nova.append(0)
    else:
        lista_nova.append(color)
        
data["color"] = lista_nova
data.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,facenumber_in_poster,plot_keywords,num_user_for_reviews,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,movie_facebook_likes
0,0.0,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,0.0,avatar|future|marine|native|paraplegic,3054.0,USA,PG-13,237000000.0,2009.0,936.0,7.9,33000
1,0.0,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,0.0,goddess|marriage ceremony|marriage proposal|pi...,1238.0,USA,PG-13,300000000.0,2007.0,5000.0,7.1,0
2,0.0,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,1.0,bomb|espionage|sequel|spy|terrorist,994.0,UK,PG-13,245000000.0,2015.0,393.0,6.8,85000
3,0.0,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,0.0,deception|imprisonment|lawlessness|police offi...,2701.0,USA,PG-13,250000000.0,2012.0,23000.0,8.5,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,0


In [27]:
def numerar(dataframe, coluna):
    # Criando a lista de nomes únicos:
    unico = []
    for classe in dataframe[coluna]:
        if classe not in unico:
            unico.append(classe)
            
    # Criando a lista de nomes alterados para números:
    reclassificado = []        
    for item in dataframe[coluna]:
        reclassificado.append(unico.index(item))
    
    # Substituindo a coluna toda do DataFrame:
    dataframe[coluna] = reclassificado
    
    # Retornando o novo DataFrame:
    return dataframe

In [28]:
numerar(data, "actor_2_name")
numerar(data, "director_name")
numerar(data, "actor_1_name")
numerar(data, "movie_title")
numerar(data, "genres")
numerar(data, "actor_3_name")
numerar(data, "language")
numerar(data, "plot_keywords")
numerar(data, "country")
numerar(data, "content_rating")
data.head()

KeyError: 'language'

In [ ]:
data.corr()

In [10]:
#k_means = cluster.KMeans(n_clusters=3)
#k_means.fit(data)

In [15]:
#plt3d = plt.figure().gca(projection="3d")
#plt3d.scatter(data.color, data.director_name, data.num_critic_for_reviews)